In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [2]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Travis Kelce,KAN,TE,11,238.7
1,Josh Jacobs,LVR,RB,11,242.4
2,Derrick Henry,TEN,RB,11,217.8
3,Patrick Mahomes,KAN,QB,11,280.8
4,Josh Allen,BUF,QB,11,277.4
...,...,...,...,...,...
419,Quintin Morris,BUF,TE,10,12.0
420,Albert Okwuegbunam,DEN,TE,5,12.0
423,James Proche,BAL,WR,9,11.3
430,Mitchell Wilcox,CIN,TE,11,10.7


### 2. Get Projections

##### 2a. My Projections

In [3]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [4]:
# Games played dictionary
page = requests.get("https://www.pro-football-reference.com/")
if page.status_code == 200:
    soup = BeautifulSoup(page.content, 'html.parser')

games_played = {'2TM' : 9}
afc = soup.find('table', id = 'AFC')
nfc  = soup.find('table', id = 'NFC')

for conference in ['AFC', 'NFC']:    
    conf = soup.find('table', id = conference)
    for row in conf.find_all('tr'):
        head = row.find_all('th', {'scope' : 'row'})
        if len(head) > 0:
            tm_name = re.sub('[^a-zA-Z]+', '', row.find('th').text)
            cols = row.find_all('td')
            gp = int(cols[0].text) + int(cols[1].text) + int(cols[2].text)
            games_played[tm_name] = gp
print(games_played)

{'2TM': 9, 'MIA': 11, 'BUF': 11, 'NYJ': 11, 'NWE': 11, 'BAL': 11, 'CIN': 11, 'CLE': 11, 'PIT': 11, 'TEN': 11, 'IND': 12, 'JAX': 11, 'HOU': 11, 'KAN': 11, 'LAC': 11, 'LVR': 11, 'DEN': 11, 'PHI': 11, 'DAL': 11, 'NYG': 11, 'WAS': 12, 'MIN': 11, 'DET': 11, 'GNB': 12, 'CHI': 12, 'TAM': 11, 'ATL': 12, 'CAR': 12, 'NOR': 12, 'SFO': 11, 'SEA': 11, 'ARI': 12, 'LAR': 11}


In [5]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}


In [6]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

In [7]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = pd.to_numeric(newerPts['Tm'].replace(games_played))
    newerPts['PPR_adj'] = newerPts['PPR'] / newerPts['GP'] * 17

    newerPts.rename(columns = {'Points' : 'Points_ESPN'}, inplace = True)
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
newerPts
newerPts.to_excel('test.xlsx')

In [8]:
newerPts['Points_ESPN'].isnull().sum()

0

In [9]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points_ESPN'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.5463055546122059
0.5164569286306578
0.5166845634229338


0      368.900000
1      374.618182
2      336.600000
3      433.963636
4      428.709091
          ...    
381     18.545455
382     18.545455
383     17.463636
384     16.536364
385     18.236364
Name: PPR_adj, Length: 386, dtype: float64